In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
#today date
today = str(date.today())
print(today)

#read csv file
df = pd.read_csv("./data/delhi-" + today + ".csv")

print(len(df))
df.head()

2020-04-11
11


District  State  Confirmed  Recovered  Deaths  Active  \
0     Central Delhi  Delhi          0          0       0       0   
1        East Delhi  Delhi          1          0       0       1   
2         New Delhi  Delhi          5          0       0       5   
3       North Delhi  Delhi          3          0       0       3   
4  North East Delhi  Delhi          1          0       0       1   

   Delta_Confirmed  Delta_Recovered  Delta_Deaths  Last_Updated_Time  
0                0                0           NaN                NaN  
1                0                0           NaN                NaN  
2                0                0           NaN                NaN  
3                0                0           NaN                NaN  
4                0                0           NaN                NaN

In [3]:
#df.fillna(0, inplace = True)
df["Confirmed"] = df["Confirmed"].apply(lambda x: None if x==0 else x)
df.head()

District  State  Confirmed  Recovered  Deaths  Active  \
0     Central Delhi  Delhi        NaN          0       0       0   
1        East Delhi  Delhi        1.0          0       0       1   
2         New Delhi  Delhi        5.0          0       0       5   
3       North Delhi  Delhi        3.0          0       0       3   
4  North East Delhi  Delhi        1.0          0       0       1   

   Delta_Confirmed  Delta_Recovered  Delta_Deaths  Last_Updated_Time  
0                0                0           NaN                NaN  
1                0                0           NaN                NaN  
2                0                0           NaN                NaN  
3                0                0           NaN                NaN  
4                0                0           NaN                NaN

In [4]:
import geopandas as gpd

#read the shp file
gdf = gpd.read_file("./admin/delhi_district/delhi_district.shp")
print(len(gdf))
gdf.head()

11


dt_code          district st_code    year  st_nm  \
0     091       North Delhi      07  2011_c  Delhi   
1     092  North East Delhi      07  2011_c  Delhi   
2     096        West Delhi      07  2011_c  Delhi   
3     093        East Delhi      07  2011_c  Delhi   
4     097  South West Delhi      07  2011_c  Delhi   

                                            geometry  
0  POLYGON ((77.20776 28.79165, 77.20880 28.78836...  
1  POLYGON ((77.29832 28.70869, 77.29521 28.69486...  
2  POLYGON ((77.17221 28.66721, 77.16650 28.65667...  
3  POLYGON ((77.29780 28.56713, 77.28457 28.59873...  
4  POLYGON ((76.95347 28.66743, 76.97968 28.65426...

In [5]:
# merge the data
mersed_df = gdf.merge(df, left_on = "district", right_on = "District", how = "left")
mersed_df.head()

dt_code          district st_code    year  st_nm  \
0     091       North Delhi      07  2011_c  Delhi   
1     092  North East Delhi      07  2011_c  Delhi   
2     096        West Delhi      07  2011_c  Delhi   
3     093        East Delhi      07  2011_c  Delhi   
4     097  South West Delhi      07  2011_c  Delhi   

                                            geometry          District  State  \
0  POLYGON ((77.20776 28.79165, 77.20880 28.78836...       North Delhi  Delhi   
1  POLYGON ((77.29832 28.70869, 77.29521 28.69486...  North East Delhi  Delhi   
2  POLYGON ((77.17221 28.66721, 77.16650 28.65667...        West Delhi  Delhi   
3  POLYGON ((77.29780 28.56713, 77.28457 28.59873...        East Delhi  Delhi   
4  POLYGON ((76.95347 28.66743, 76.97968 28.65426...  South West Delhi  Delhi   

   Confirmed  Recovered  Deaths  Active  Delta_Confirmed  Delta_Recovered  \
0        3.0          0       0       3                0                0   
1        1.0          0       0       1                0                0   
2        2.0          0       0       2                0                0   
3        1.0          0       0       1                0                0   
4        3.0          0       0       3                0                0   

   Delta_Deaths  Last_Updated_Time  
0           NaN                NaN  
1           NaN                NaN  
2           NaN                NaN  
3           NaN                NaN  
4           NaN                NaN

In [6]:
import json

# read data to json
json_df = json.loads(mersed_df.to_json())

#convert the string like object
json_data = json.dumps(json_df)
#json_data

In [7]:
from bokeh.io import curdoc, output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import HoverTool, PanTool, WheelZoomTool, UndoTool, RedoTool, ResetTool
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['RdYlGn'][11]

#Reverse color order so that dark blue is highest obesity.
#palette = palette[::-1]

maxValue = int(df["Confirmed"].max())

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors. Input nan_color.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = maxValue, nan_color = '#FFFFFF')
#Create color bar. 
color_bar = ColorBar(color_mapper = color_mapper, label_standoff = 8, width = 25, height = 580, border_line_color=None,
                     location = (0, 0), orientation = 'vertical')


#Create figure object.
p = figure(title = df.State[0] + ' Spread of Covid-19 Confirmed Cases, ('+ today +'). (Sujeet Singh)', plot_height = 650 , 
           plot_width = 980, background_fill_color = "#aaaaaa", background_fill_alpha = 0.4)

p.title.text_font = "italic" #"helvetica"
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = "olive"
# p.background_fill_color = "beige"

#Add patch renderer to figure. 
patch = p.patches('xs','ys', source = geosource, fill_color = {'field' :'Confirmed', 'transform' : color_mapper},
                  line_color = '#d95f0e', line_width = 0.35, fill_alpha = 1, hover_fill_color="#aaaaaa")

p.add_tools(HoverTool(tooltips=[ ('District ',' @district'), ('Confirmed Cases ',' @Confirmed')], renderers=[patch]))


#Specify figure layout.
p.add_layout(color_bar, 'left')

#Display figure inline in Jupyter Notebook.
output_notebook()
curdoc().add_root(p)
#Display figure.
show(p)
print("Completed")

Loading BokehJS ...

Completed
